# Data Preprocessing

* 필요한 라이브러리 import

In [1]:
import os
import pandas as pd
from tqdm import trange
import shutil

* AI Hub 데이터셋을 저장한 베이스 폴더와 최상위 폴더 5개의 리스트를 미리 선언

In [2]:
BASEPATH='E:/한국어 음성데이터'
TOP_DIRS=[
    'KsponSpeech_01',
    'KsponSpeech_02',
    'KsponSpeech_03',
    'KsponSpeech_04',
    'KsponSpeech_05'
]

* (철자전사) / (발음전사) 중 발음전사를 가져오는 함수

In [3]:
def bracket_filter(sentence):
    new_sentence = str()
    flag = False
    
    for ch in sentence:
        if ch == '(' and flag == False: 
            flag = True
            continue
        if ch == '(' and flag == True:
            flag = False
            continue
        if ch != ')' and flag == False:
            new_sentence += ch
    return new_sentence

* 특수문자 필터링

In [4]:
def special_filter(sentence):
    SENTENCE_MARK = ['?', '!']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', ',', '.']
    import re
    new_sentence = str()
    
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            # o/, n/ 등 처리
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
                continue 

        if ch == '#': 
            new_sentence += '샾'
        elif ch not in EXCEPT: 
            new_sentence += ch
            
    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence

* 위의 두 함수를 불러서 문자열을 필터링

In [5]:
def sentence_filter(raw_sentence):
    return special_filter(bracket_filter(raw_sentence))

In [ ]:
for t_dir in TOP_DIRS:
    dir_list = os.listdir(os.path.join(BASEPATH, t_dir))
    
    for idx in trange(len(dir_list)):
        file_list = os.listdir(os.path.join(BASEPATH, t_dir, dir_list[idx]))
        
        for file in file_list:
            if file.endswith('_new.txt'):
                continue
                
            elif file.endswith('.txt'):
                f = open(os.path.join(BASEPATH, t_dir, dir_list[idx], file), mode="r")
                raw_sentence = f.readline()
                new_sentence = sentence_filter(raw_sentence)
                f.close()
                
                f = open(os.path.join('E:/AIHub', file), mode="w")
                f.write(new_sentence)
                f.close()
                
            elif file.endswith('.pcm'):
                shutil.copy(os.path.join(BASEPATH, t_dir, dir_list[idx], file), 'E:/AIHub')
                
            else:
                print(file)

  0%|                                                                                          | 0/124 [00:00<?, ?it/s]